Packages

In [40]:
!pip3 install psycopg2
!pip3 install neo4j
!pip3 install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


In [9]:
import pandas as pd
import json
import psycopg2
from neo4j import GraphDatabase
from decimal import Decimal
from dotenv import load_dotenv
import os

load_dotenv()

True

#### Connect to Database `nourish@awesome-hw.sdsc.edu`

In [10]:
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=5432,
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)

with conn.cursor() as cursor:
    cursor.execute("SELECT version();")
    print(cursor.fetchone()[0])

    cursor.execute("SELECT COUNT(*) FROM INFORMATION_SCHEMA.TABLES")
    print(cursor.fetchall()[0][0], "tables found in the database.")

PostgreSQL 12.18 (Ubuntu 12.18-0ubuntu0.20.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0, 64-bit
370 tables found in the database.


#### Create Postgres tables and export data to JSON files

In [11]:
# Create Postrgress tables (Only OWNER can run this - swhoyle)
print("Creating postgress tables...")
with open ("../scripts/create_postgres_tables.sql", "r") as f:
    sql = f.read()
with conn.cursor() as cursor:
    cursor.execute(sql)
    conn.commit()

Creating postgress tables...


In [12]:
# Entity Table ETL
print("Inserting data into entity tables...")
with open ("../scripts/etl_entity_tables.sql", "r") as f:
    sql = f.read()
with conn.cursor() as cursor:
    cursor.execute(sql)
    conn.commit()

Inserting data into entity tables...


In [13]:
# Entity Relationships ETL
print("Inserting data into entity_relationships table...")
with open ("../scripts/etl_entity_relationships.sql", "r") as f:
    sql = f.read()
with conn.cursor() as cursor:
    cursor.execute(sql)
    conn.commit()

Inserting data into entity_relationships table...


In [14]:
# Export entity tables to JSON files
print("Exporting entity tables to JSON files...")
entities = ["state", "county", "city", "community", "zipcode", "blockgroup", "business"]
for entity in entities:

    with conn.cursor() as cursor:
        cursor.execute(f"SELECT * FROM entity_{entity};")
        columns = [desc[0] for desc in cursor.description]
        rows = cursor.fetchall()
        results=[]
        for row in rows:
            row_dict = {}
            for col, val in zip(columns, row):
                if isinstance(val, Decimal):
                    val = float(val)  # or str(val) if you prefer
                row_dict[col] = val
            results.append(row_dict)

    with open(f"../data/{entity}.json", "w") as f:
        json.dump(results, f, indent=4)

    print(f"-- {entity}.json exported.")

Exporting entity tables to JSON files...
-- state.json exported.
-- county.json exported.
-- city.json exported.
-- community.json exported.
-- zipcode.json exported.
-- blockgroup.json exported.
-- business.json exported.


In [16]:
# Export relationship table to JSON file
print("Exporting relationship table to JSON file...")
with conn.cursor() as cursor:
    cursor.execute("SELECT * FROM entity_relationships;")
    columns = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()
    results = [dict(zip(columns, row)) for row in rows]
    
with open("../data/relationships.json", "w") as f:
    json.dump(results, f, indent=4)

print(f"-- relationships.json exported.")

Exporting relationship table to JSON file...
-- relationships.json exported.


#### Connect to neo4j

In [55]:
uri = "bolt://awesome-compute.sdsc.edu:7687"
username = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(uri, auth=(username, password))
with driver.session() as session:
    result = session.run("MATCH (n) RETURN count(n) AS cnt")
    print(result.single()["cnt"], "nodes in the graph")


247253 nodes in the graph


#### Load Entities to neo4j

In [176]:
print("Ingesting entity data into Neo4j...")

# Delete existing nodes:
print("\nDeleting existing nodes...")
for entity in entities:
    
    # Delete existing nodes of this entity type
    entity_label = entity.capitalize()

    with driver.session() as session:
        existing_count = session.run(f"MATCH (n:{entity_label}) RETURN count(n) AS cnt").single()["cnt"]
        session.run(f"MATCH (n:{entity_label}) DETACH DELETE n")
    print(f"-- {existing_count} {entity_label} nodes deleted.")


print(f"\nCreating nodes...")
for entity in entities:

    with open(f"../data/{entity}.json", "r") as f:
        data = json.load(f)

    # Delete existing nodes of this entity type
    entity_label = entity.capitalize()

    # Create nodes in neo4j for each entity
    with driver.session() as session:
        for record in data:
            props = ", ".join([f"`{key}`: $`{key}`" for key in record.keys()])
            query = f"CREATE (n:{entity_label} {{{props}}})"
            session.run(query, **record)
    print(f"-- {len(data)} {entity_label} nodes created.")

Ingesting entity data into Neo4j...

Deleting existing nodes...
-- 6 Business nodes deleted.

Creating nodes...
-- 2000 Business nodes created.


#### Query Entity data in neo4j

In [173]:
# For each entity, show top 3 nodes
for entity in entities:
    entity_label = entity.capitalize()
    entity_count = driver.session().run(f"MATCH (n:{entity_label}) RETURN count(n) AS cnt").single()["cnt"]
    node_text = "nodes" if entity_count != 1 else "node"
    print(f"{entity_label}: {entity_count} {node_text} found.")
    with driver.session() as session:
        result = session.run(f"MATCH (n:{entity_label}) RETURN n LIMIT 3")
        i = 0
        for record in result:
            if i > 1:
                print("...")
            print(record.data())
            i += 1


    print("")

State: 1 node found.
{'n': {'code': 'CA', 'name': 'California', 'id': 1}}

County: 1 node found.
{'n': {'name': 'San Diego', 'id': 37}}

City: 54 nodes found.
{'n': {'name': 'Jacumba', 'id': 54}}
{'n': {'name': 'Camp Pendleton North', 'id': 42}}
...
{'n': {'name': 'Bostonia', 'id': 40}}

Community: 229 nodes found.
{'n': {'name': 'Midtown', 'id': 60}}
{'n': {'name': 'Linda Vista', 'id': 55}}
...
{'n': {'name': 'Eastlake Trails', 'id': 284}}

Zipcode: 99 nodes found.
{'n': {'zipcode': '91935'}}
{'n': {'zipcode': '92140'}}
...
{'n': {'zipcode': '92109'}}

Blockgroup: 2085 nodes found.
{'n': {'n_2021HealthCare': 0.0, 'n_2023Manufacturing': 112.0, 'countyfp': '073', 'n_2023TotalCrimeIndex': 101.0, 'geom': '0106000020B608000001000000010300000001000000B8000000E064394018EE5741C0EFDC040C573C4180F11783FDED5741C0163E200E573C41209D9F90EEED574140A5C34D0F573C41B0AB3A80E5ED574180E89B0410573C41C0837D85CEED574100BABA70DD563C411066EA43CEED574140F11413DC563C41B000EA28CCED574180A610B0CE563C41302AD518C7ED

#### Add entity relationships to neo4j

In [ ]:
with open("../data/relationships.json", "r") as f:
    data = json.load(f)

print("Deleting all existing relationships (edges) in Neo4j...")
for entity in entities:
    entity_label = entity.capitalize()
    with driver.session() as session:
        existing_count = session.run(f"MATCH (n:{entity_label})-[r]->() RETURN count(r) AS cnt").single()["cnt"]
        session.run(f"MATCH (n:{entity_label})-[r]->() DELETE r")
    print(f"-- {existing_count} relationships deleted for {entity_label}.")


print("\nCreating relationships (edges) in Neo4j...")

df = pd.DataFrame(data)
relationship_types = df.groupby(["entitytype1", "entitytype2", "predicate"]).size().reset_index(name="count")
for index, row in relationship_types.iterrows():
    et1 = row["entitytype1"].capitalize()
    et2 = row["entitytype2"].capitalize()
    predicate = row["predicate"]
    count = row["count"]
    space = " " * (35 - (len(et1) + len(et2) + len(predicate)))

    filtered_df = df[
        (df["entitytype1"] == row["entitytype1"]) & 
        (df["entitytype2"] == row["entitytype2"]) & 
        (df["predicate"] == row["predicate"])
    ]

    entity1_att = "name"
    entity2_att = "name"


    if et1 == "Business":
        entity1_att = "id"
    if et2 == "Blockgroup":
        entity2_att = "ctblockgroup"
        
    for _, drow in filtered_df.iterrows():
        entity1 = drow["entity1"]
        entity2 = drow["entity2"] 

        with driver.session() as session:
            query = (
                f"MATCH (a:{et1} {{{entity1_att}: $entity1}}), "
                f"(b:{et2} {{{entity2_att}: $entity2}}) "
                f"CREATE (a)-[r:{predicate}]->(b)"
            )
            session.run(query, entity1=entity1, entity2=entity2)

    print(f"{et1} -[{predicate}]-> {et2}: {count} relationships created")

print(f"\n{len(data)} total relationships created.")

Deleting all existing relationships (edges) in Neo4j...
-- 2032 relationships deleted for Business.

Creating relationships (edges) in Neo4j...
Business -[contained_in]-> Blockgroup: 2000 relationships created


ValueError: invalid literal for int() with base 10: 'Felicity'

In [27]:
# View all business -> block group relationships
with driver.session() as session:
    result = session.run(
        "MATCH (b:Business)-[r]->(bg:Blockgroup) "
        "RETURN b.id AS business_id, type(r) AS relationship, bg.name AS blockgroup_name "
        "LIMIT 10"
    )
    for record in result:
        print(record.data())

{'business_id': 19527, 'relationship': 'contained_in', 'blockgroup_name': None}


#### Query Entity Relationships

In [149]:
# View communities within "San Diego" city
with driver.session() as session:
    result = session.run(
        "MATCH (c:Community)-[:contained_in]->(ci:City) "
        "WHERE ci.name = $city_name "
        "RETURN c.name AS community_name LIMIT 5",
        city_name="San Diego"
    )
    print("Communities within San Diego city:")
    for record in result:
        print(f"- {record['community_name']}")



# View cities nearby "Poway" city
with driver.session() as session:
    result = session.run(
        "MATCH (c1:City)-[:nearby]->(c2:City) "
        "WHERE c1.name = $city_name "
        "RETURN c2.name AS nearby_city LIMIT 5",
        city_name="Poway"
    )
    print("\nCities nearby Poway:")
    for record in result:
        print(f"- {record['nearby_city']}")

# View communities adjacent to "College West" community
with driver.session() as session:
    result = session.run(
        "MATCH (c1:Community)-[:adjacent_to]->(c2:Community) "
        "WHERE c1.name = $community_name "
        "RETURN c2.name AS adjacent_to LIMIT 5",
        community_name="College West"
    )
    print("\nCommunities adjacent to College West:")
    for record in result:
        print(f"- {record['adjacent_to']}")

Communities within San Diego city:
- Burlingame
- Sherman Heights
- Rancho Bernardo
- Rancho Bernardo
- Rancho Bernardo

Cities nearby Poway:
- Escondido
- San Marcos
- Santee

Communities adjacent to College West:
- Grantville
- Kensington
- El Cerrito
- College East
- College East


In [151]:
# View all cities in "San Diego", sortedd by number of communities (include 0 communities)
with driver.session() as session:
    result = session.run(
        "MATCH (ci:City) "
        "OPTIONAL MATCH (ci)<-[:contained_in]-(c:Community) "
        "WHERE ci.name = $city_name "
        "RETURN ci.name AS city_name, count(c) AS community_count "
        "ORDER BY community_count DESC",
        city_name="San Diego"
    )
    print("\nCities in San Diego sorted by number of communities:")
    for record in result:
        print(f"- {record['city_name']}: {record['community_count']} communities")


Cities in San Diego sorted by number of communities:
- San Diego: 157 communities
- Lemon Grove: 0 communities
- Borrego Springs: 0 communities
- Julian: 0 communities
- Imperial Beach: 0 communities
- Hidden Meadows: 0 communities
- Jamul: 0 communities
- Bonsall: 0 communities
- Jacumba: 0 communities
- Camp Pendleton North: 0 communities
- Bostonia: 0 communities
- Harbison Canyon: 0 communities
- Boulevard: 0 communities
- Winter Gardens: 0 communities
- Valley Center: 0 communities
- Spring Valley: 0 communities
- Santee: 0 communities
- Rancho Santa Fe: 0 communities
- Ramona: 0 communities
- Rainbow: 0 communities
- Pala: 0 communities
- La Presa: 0 communities
- Lakeside: 0 communities
- San Marcos: 0 communities
- Rancho San Diego: 0 communities
- Chula Vista: 0 communities
- Carlsbad: 0 communities
- La Mesa: 0 communities
- Vista: 0 communities
- Solana Beach: 0 communities
- San Diego Country Estates: 0 communities
- Escondido: 0 communities
- Lake San Marcos: 0 communitie

In [ ]:
# Show all cities and their nearby relationships
with driver.session() as session:
    result = session.run(
        "MATCH (c1:City)-[:nearby]->(c2:City) "
        "RETURN c1.name AS city, collect(c2.name) AS nearby_cities "
        "LIMIT 5"
    )
    print("\nCities and their nearby relationships:")
    for record in result:
        print(f"- {record['city']}: {', '.join(record['nearby_cities'])}")